In [ ]:
import configobj
import cPickle
from crate import client
import glob
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt
from teams import nba_teams
team_keys = nba_teams

config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

In [ ]:
files = glob.glob("/Users/smacmullin/sports/2015models/results/*")
for f in files:
    with open(f,'rb') as fp:
        results = cPickle.load(fp)
        
    for game in results:
        
        row = [game["GameId"], game["Spread"], game["StdSpread"], game["OU"], game["StdOU"], game["Games"]]
        query = '''INSERT INTO nba.predictions ("GameId", "Spread", "SpreadStdev", "OU", "OUStdev", "N")
                VALUES (?,?,?,?,?,?)'''
        cursor.execute(query, row)

In [ ]:
sql = '''select nba.results."GameId", (nba.results."AwayScore" - nba.results."HomeScore") as "Result", 
nba.predictions."Spread" as "Prediction", nba.lines."HomeSpread" as "Offered",
nba.predictions."Spread" - (nba.results."AwayScore" - nba.results."HomeScore") as "P-R",
nba.lines."HomeSpread" - (nba.results."AwayScore" - nba.results."HomeScore") as "O-R"
FROM nba.results, nba.predictions, nba.lines
WHERE nba.results."GameId"=nba.predictions."GameId" AND nba.results."GameId"=nba.lines."GameId"
AND nba.predictions."N"=6'''

df = pd.read_sql(sql, connection)
print df.head()

result = df["Result"].values
prediction = df["Prediction"].values
offered = df["Offered"].values

bins = np.linspace(-50,50,20)
plt.hist(result-prediction,bins,histtype='step',color='b',label="Model")
plt.hist(result-offered,bins,histtype='step',color='g',label="Line")
plt.xlabel("Actual -  Predicted Spread")
plt.legend()
plt.show()

win_counter = 0.
for r,p,o in zip(result, prediction, offered):
    if np.abs((p-r)) < np.abs((o-r)):
    #if np.abs((np.random.normal(0,10))) < np.abs(o-r):
        win_counter+=1.
print win_counter/len(result)

In [ ]:
# get each game id
sql = '''select DISTINCT nba.results."GameId" from nba.results'''

cursor.execute(sql)
games = [row[0] for row in cursor]

wins = 0.0
ngames = 0.0

predicted_all = []
offered_all = []
result_all = []

for game in games:
    sql = '''select nba.results."GameId", (nba.results."AwayScore" - nba.results."HomeScore") as "Result", 
            nba.predictions."Spread" as "Prediction", nba.lines."HomeSpread" as "Offered",
            nba.predictions."Spread" - (nba.results."AwayScore" - nba.results."HomeScore") as "P-R",
            nba.lines."HomeSpread" - (nba.results."AwayScore" - nba.results."HomeScore") as "O-R",
            nba.predictions."N"
            FROM nba.results, nba.predictions, nba.lines
            WHERE nba.results."GameId"=nba.predictions."GameId" AND nba.results."GameId"=nba.lines."GameId"
            AND nba.results."GameId" = '%s' 
            ORDER BY nba.predictions."N" '''%game
    df = pd.read_sql(sql, connection)
    predicted = np.average(df['Prediction'].values)
    predicted_std = np.std(df['Prediction'].values)
    offered = np.average(df['Offered'].values)
    result = np.average(df['Result'].values)
#     print df
#     print predicted
#     print offered
#     print result
    
    if True: # select a subset of games
    
        predicted_all.append(predicted)
        offered_all.append(offered)
        result_all.append(result)
        
        ngames += 1.0
        if ((result < offered) and (predicted < offered)) or ((result > offered) and (predicted > offered)):
            wins += 1.0


print wins
print ngames
print wins/ngames

bins = np.linspace(-50,50,20)
plt.hist(np.asarray(result_all)-np.asarray(predicted_all),bins,histtype='step',color='b',label="Model")
plt.hist(np.asarray(result_all)-np.asarray(offered_all),bins,histtype='step',color='g',label="Line")
plt.xlabel("Actual -  Predicted Spread")
plt.legend()
plt.show()

In [ ]:
print np.std(result-offered)
print np.random.normal(0,11.6)